In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [2]:
%matplotlib widget

### Discharge data

In [42]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [43]:
step_cal_dir = r'/data/new_trap/20210622/bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=300)

100%|██████████| 280/280 [00:18<00:00, 10.26it/s]

580  files in folder
280  files loaded


In [44]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 280
[0.08190331] 9.996283880412916
1 / 280
[0.08190909] 9.996281304225382
2 / 280
[0.08189878] 9.996281537733502
3 / 280
[0.08188683] 9.996288002004473
4 / 280
[0.08188301] 9.996283855239406
5 / 280
[0.0818812] 9.996280896041402
6 / 280
[0.08188132] 9.996280781109695
7 / 280
[0.08187602] 9.996281466488083
8 / 280
[0.08187166] 9.996283836062767
9 / 280
[0.08186824] 9.99628049649982
10 / 280
[0.08185259] 9.99628122742368
11 / 280
[0.08184229] 9.996278292760515
12 / 280
[0.08184375] 9.9962872679154
13 / 280
[0.08185011] 9.996282311106809
14 / 280
[0.08185688] 9.996282232848417
15 / 280
[0.08187448] 9.99628608262862
16 / 280
[0.0818733] 9.996285608676523
17 / 280
[0.08187297] 9.996284865507036
18 / 280
[0.08186803] 9.996282915362116
19 / 280
[0.08185791] 9.996282562713075
20 / 280
[0.08185112] 9.996280282430336
21 / 280
[0.08186028] 9.996284879184568
22 / 280
[0.08187175] 9.99628085579499
23 / 280
[0.08186014] 9.996283020975962
24 / 280
[0.08186719] 9.996287090219038
25 / 280
[0.081872

205 / 280
[0.0819037] 9.996285364134158
206 / 280
[0.0819198] 9.996282926699658
207 / 280
[0.08191724] 9.996286803289863
208 / 280
[0.08191292] 9.996282726737947
209 / 280
[0.08190771] 9.99627836823018
210 / 280
[0.08189533] 9.996286220251093
211 / 280
[0.08189578] 9.996282530728573
212 / 280
[0.08189087] 9.996279481465574
213 / 280
[0.0818991] 9.996283213603508
214 / 280
[0.08190371] 9.996283979268693
215 / 280
[0.08190964] 9.996278217408733
216 / 280
[0.08190812] 9.996281835959412
217 / 280
[0.08189192] 9.996278896802057
218 / 280
[0.08189368] 9.996275926505898
219 / 280
[0.08190671] 9.996279128442351
220 / 280
[0.08179691] 9.99627468897426
221 / 280
[0.08182042] 9.99627516703645
222 / 280
[0.08180523] 9.99627521228304
223 / 280
[0.08181146] 9.99627824130788
224 / 280
[0.08183009] 9.996278267050043
225 / 280
[0.08180829] 9.996275926682001
226 / 280
[0.08182235] 9.996279022607434
227 / 280
[0.08182145] 9.996281920292775
228 / 280
[0.08182219] 9.99628283021442
229 / 280
[0.08184658] 9.

In [45]:
scale = np.abs(np.mean(amps3[180:200]))
scaleY = scale*-1.0502
print("Scale is ", scaleY)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)
print("voltage: ", voltage)

Scale is  -3.54295075496134e-05
voltage:  9.996282495738035


In [39]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots()
# scale = 326.26

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

# ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-1.2,6.5), xlim=(160,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp(i):
    
    fname = r'/data/new_trap/20210622/bead1/Neutrality2/Relaxation_20210708/200V/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    drive_freq = 71
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 'sigma': 0.1,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
                  'limit_A': [-100, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y3)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = 142
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': np.pi/2, 'sigma': 0.1,
              'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
              'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
              'limit_A': [-100, 100], 
              'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y3)[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[3], m2_tmp.values[0]

In [52]:
i=0
get_amp(i)/scaleY3

array([-0.47541423, -0.92002077,  0.14214003])

In [10]:
i = 0
fname = r'/data/new_trap/20210622/bead1/Neutrality2/Relaxation_20210708/200V_2/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_200 = np.std(bb.electrode_data[0])*np.sqrt(2)*200
voltage1_200 = np.std(bb.electrode_data[1])*np.sqrt(2)*200
bias2 = np.mean(bb.electrode_data[0])*200
print("voltages, bias: ", voltage0_200, voltage1_200)

voltages, bias:  199.91493035973804 0.006001911993640894


In [11]:
voltage, voltage0_200

(9.996282495738035, 199.91493035973804)

In [53]:
amps3_data_200V_1 = np.array(Parallel(n_jobs=20)(delayed(get_amp)(i) for i in tqdm(range(1996))))

100%|██████████| 1996/1996 [01:02<00:00, 31.83it/s]


In [49]:
amps3_data_200V_2 = np.array(Parallel(n_jobs=20)(delayed(get_amp)(i) for i in tqdm(range(18304))))

100%|██████████| 18304/18304 [06:31<00:00, 46.80it/s]


### Plot raw data

In [16]:
##
i=0; j=1
_,ax = plt.subplots()
dat = amps_data_200V_2
ax.scatter(dat[::4,i], dat[::4,j],label='positive electrode')
ax.scatter(dat[2::4,i], dat[2::4,j], label='negative electrode')
ax.set(xlabel='amplitude', ylabel='noise parameter')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
amps_all = list(np.array(amps_data_200V_1))
amps_all += list(np.array(amps_data_200V_2))
# amps_all += list(np.array(amps_data_200V_3))
amps_all = np.array(amps_all)

In [55]:
amps3_all = list(np.array(amps3_data_200V_1))
amps3_all += list(np.array(amps3_data_200V_2))
# amps_all += list(np.array(amps_data_200V_3))
amps3_all = np.array(amps3_all)

In [21]:
data0 = amps_all[:,0]/scaleY*voltage/voltage0_200

dataPlus=np.array(list(data0[0::4]))
dataMinus=np.array(list(data0[2::4]))
data_A = dataPlus+factor*dataMinus


_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='Y-axis/, plus, 200V, XZ-spin-46kHz, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='Y-axis, minus, 200V, XZ-spin-46kHz, with-height-fb')
ax.plot(np.arange(0,len(dataPlus)), np.arange(0,len(dataPlus))*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))
print(np.mean(data_A), np.std(data_A/np.sqrt(len(data_A))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  -0.0001139107881377562 std:  6.583141056141887e-05
mean:  4.3564407080790014e-05 std:  6.578183328163265e-05
-8.176166429430102e-05 8.214022527946122e-05


In [57]:
data0 = amps3_all[:,0]/scaleY3*voltage/voltage0_200

dataPlus=np.array(list(data0[0::4]))
dataMinus=np.array(list(data0[2::4]))
data_A = dataPlus+factor*dataMinus


_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='Y-axis/, plus, 200V, XZ-spin-46kHz, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='Y-axis, minus, 200V, XZ-spin-46kHz, with-height-fb')
ax.plot(np.arange(0,len(dataPlus)), np.arange(0,len(dataPlus))*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))
print(np.mean(data_A), np.std(data_A/np.sqrt(len(data_A))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  -0.00025347367588278896 std:  0.00016021327982432572
mean:  -0.0001486791949770029 std:  0.0001523738890374019
-0.0004021528708597918 0.00022241877020530334


In [66]:
N = len(dataMinus)

data_shifted = np.array([(dataMinus[i]+dataMinus[i-1])/2 for i in range(1,N)])
data3_A_shifted = data_shifted+factor*dataPlus[:-1]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_A))*2*10/3600, data_A, label='200Vpp-z-plus, new trap')
ax.scatter(np.arange(len(data3_A_shifted))*2*10/3600, data3_A_shifted, label='200Vpp-z, A parameter corrected', alpha=0.6)

ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))
print('1st harmonic, A-shifted: mean, std: ', np.mean(data_A_shifted), np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  -0.0004021528708597918 0.0002224187702053033
1st harmonic, A-shifted: mean, std:  -7.242485672168804e-05 8.05042847695174e-05


In [23]:
print(len(dataMinus), len(dataMinus))
dataPlus_noise=amps_data_200V_2[::4,0]
dataMinus_noise=amps_data_200V_2[2::4,0]
thresh = 0.002
factor = 1
amps_data_200V_after_cuts = np.array([dataMinus[i]*factor+dataPlus[i] for i in range(len(dataMinus)) if (dataPlus_noise[i]<thresh)&(dataMinus_noise[i]<thresh)])

5075 5075


IndexError: index 4576 is out of bounds for axis 0 with size 4576

In [63]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

data_hist = dataPlus+factor*dataMinus
range_pram = 0.03
data_hist = np.array(list(data_A_shifted))
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_hist, bins=31, range=(-range_pram, range_pram), label='200V-x, xz-spin-100V')
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]
# bins_fit = bins[:-1]
for i,ee in enumerate([entries]):
    x = bins_fit
    y= ee

    n = len(x)                          #the number of data
    mean = sum(x*y)/n                   #note this correction
    sigma = sum(y*(x-mean)**2)/n        #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], "cov: ", np.sqrt(pcov)[1,1])
    print("mean: ", popt[2]/np.sqrt(len(data_hist)), "std: ", popt[2])

mean:  -5.323361582904662e-06 cov:  6.261427902426584e-05
mean:  7.980263186253493e-05 std:  0.005684502291740081


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sqrt


In [38]:
_,ax = plt.subplots()
ax.errorbar(bins_fit, entries, yerr=np.sqrt(entries), fmt='o', label='data')
ax.plot(x,gaus(x,*popt),'r--',label='fit')
ax.legend()
print("mean: ", popt[1], "std: ", np.sqrt(pcov)[1,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  7.024302531654061e-06 std:  7.787235043010116e-05


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in sqrt
  """


In [33]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp_single(i):
    
    fname = r'/data/new_trap/20210622/bead1/Single_electrode/Y_electrode_plus_10V_71hZ/Discharge_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20210622/bead1/Single_electrode/Y_electrode_minus_10V_71Hz/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 'sigma': 0.1,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
                  'limit_A': [-100, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [34]:
amps_plus_y = np.array(Parallel(n_jobs=20)(delayed(get_amp_single)(i) for i in tqdm(range(20))))

100%|██████████| 20/20 [00:00<00:00, 2013.10it/s]


In [35]:
factor = np.mean(amps_plus_y)/np.mean(amps_minus_y)
factor

0.7379676666741903

In [20]:
factor = 0.7379676666741903

In [102]:
print("mean: ", popt_minus[1]*factor+popt_plus[1])
print("std: ", np.sqrt(factor*factor*pcov_minus[1,1]+pcov_plus[1,1]))

mean:  -9.040221284622362e-06
std:  5.783918100228764e-05


In [163]:
print("mean: ", popt_minus[1]*factor+popt_plus[1])
print("std: ", np.sqrt(factor*factor*pcov_minus[1,1]+pcov_plus[1,1]))

mean:  -3.5459721632706974e-05
std:  4.7112576364709966e-05


In [65]:
np.save("tmp", data_A_shifted)
# data_A_shifted